In [1]:
import pandas as pd
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests 
from bs4 import BeautifulSoup
# from env_miatta import github_token, github_username
import time
import re
import nltk
from nltk.tokenize import ToktokTokenizer
import numpy as np

In [2]:
# Aquire data using methods described above.
words_df = pd.read_json('data2.json')
words_df.language.value_counts()

JavaScript          31
Python              15
Java                11
HTML                 7
C                    6
Jupyter Notebook     5
Shell                5
Ruby                 4
C++                  3
CSS                  3
Go                   3
TypeScript           3
SCSS                 2
PowerShell           1
Kotlin               1
Bicep                1
HCL                  1
Vim script           1
Objective-C          1
Vue                  1
Name: language, dtype: int64

In [4]:
def tokenize(text):
    """
    Tokenizes the words in the input string.
    """
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def clean(text: str) -> list: 
    """
    Cleans up the input text data.
    """
    text = (text.encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
    
    words = re.sub(r'[^\w\s]', ' ', text).split()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

def nlp_wrangle():
    """
    Performs data wrangling for natural language processing (NLP) tasks.
    Returns a processed DataFrame for NLP analysis.
    """
    # Load data from JSON file
    df = pd.read_json('data2.json')
    
    # Tokenize and clean contents
    df['clean_contents'] = df.readme_contents.apply(tokenize).apply(' '.join)
    df['clean_contents'] = df.clean_contents.apply(clean).apply(' '.join)

    # Words to remove
    words_to_remove = ["http", "com", "124", "www", "1", "github", "top", "go", "android", "content", "table",
                       "107", "markdown", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "md"]

    # Remove specific words from clean_contents
    for word in words_to_remove:
        df['clean_contents'] = df['clean_contents'].str.replace(word, '')

#     # Sentiment analysis
#     sia = nltk.sentiment.SentimentIntensityAnalyzer()
#     df['sentiment'] = df['clean_contents'].apply(lambda doc: sia.polarity_scores(doc)['compound'])

    # Add message_length and word_count columns
    df['message_length'] = df['clean_contents'].str.len()
    df['word_count'] = df.clean_contents.apply(clean).apply(len)

    # Keep only top languages and assign others to 'Other'
    languages_to_keep = ['JavaScript', 'Python', 'Java', 'TypeScript', 'HTML']
    df['language'] = np.where(df['language'].isin(languages_to_keep), df['language'], 'Other')

    # Filter DataFrame based on conditions
    df = df.loc[(df['word_count'] <= 10000) & (df['message_length'] <= 60000)]

    return df

In [5]:
words_df_w = nlp_wrangle()

In [6]:
words_df_w

,repo,language,readme_contents,clean_contents,message_length,word_count
0,TheAlgorithms/Python,Python,"<div align=""center"">\n<!-- Title: -->\n <a hr...",div align center title href thealrithms img...,1801,284
1,apache/flink,Java,# Apache Flink\n\nApache Flink is an open sour...,apache flink apache flink open source stream p...,3271,463
2,forezp/SpringCloudLearning,Java,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...,blog csdn net forezp article detail blog cs...,4825,635
3,learn-co-students/python-dictionaries-readme-d...,Other,\n# Dictionaries \n\n### Introduction\n\nAfter...,dictionary introduction introducing working li...,6115,850
4,angular/angular-phonecat,JavaScript,# AngularJS Phone Catalog Tutorial Application...,angularjs phone catalog tutorial application o...,7533,1083
...,...,...,...,...,...,...
109,lin-xin/vue-manage-system,Other,"{""payload"":{""allShortcutsEnabled"":false,""fileT...",payload allshortcutsenabled false filetree ite...,18414,2846
110,Significant-Gravitas/Auto-GPT,Python,"{""payload"":{""allShortcutsEnabled"":false,""fileT...",payload allshortcutsenabled false filetree ite...,34078,5072
112,namndwebdev/tang-crush,JavaScript,"{""payload"":{""allShortcutsEnabled"":false,""fileT...",payload allshortcutsenabled false filetree ite...,6024,859
113,learn-co-students/javascript-arrays-lab-bootca...,JavaScript,"{""payload"":{""allShortcutsEnabled"":false,""fileT...",payload allshortcutsenabled false filetree ite...,10055,1343


In [16]:
a= set(range(0,118)) - set(list(words_df_w.index))
# a = list(a)
a

{34,
 39,
 41,
 42,
 45,
 52,
 53,
 54,
 59,
 69,
 71,
 78,
 80,
 86,
 90,
 92,
 97,
 111,
 114,
 116,
 117}

In [20]:
words_df_temp = words_df[words_df.index == 41]


In [21]:
words_df_temp.readme_contents

41    {"payload":{"allShortcutsEnabled":false,"fileT...
Name: readme_contents, dtype: object

In [ ]:
cleaned_list = []

def clean(text: str) -> list: 
    """
    Cleans up the input text data.
    """
    text = (text.encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
    
    words = re.sub(r'[^\w\s]', ' ', text).split()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

for content in words_df.readme_contents:
    cleaned_text = clean(content)
    cleaned_list.append(cleaned_text)

In [ ]:
cleaned_list

In [ ]:
se = pd.Series(cleaned_list)
words_df['cleaned_text'] = se.values


In [ ]:
words_df